#  The Battle of Neighborhoods - Final Assignment !!

In the city of Toronto, An enreprenure is  looking to open an American restaurant !!

The objective of this capstone project is to find the most suitable location for the entrepreneur to open a new American Restaurant in Toronto, Canada. By using data science methods and tools along with machine learning algorithms such as clustering, this project aims to provide solutions to answer the business question: In Toronto, if an entrepreneur wants to open an  American Restaurant, where should they consider opening it!!





Upload Libraries Required

In [2]:
import requests
import pandas as pd
import numpy as np
import folium
!conda install -c anaconda xlrd --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - xlrd


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.24  |                0         133 KB  anaconda
    certifi-2020.6.20          |           py36_0         160 KB  anaconda
    openssl-1.1.1g             |       h7b6447c_0         3.8 MB  anaconda
    xlrd-1.2.0                 |             py_0         108 KB  anaconda
    ------------------------------------------------------------
                                           Total:         4.2 MB

The following packages will be UPDATED:

    ca-certificates: 2020.6.20-hecda079_0     conda-forge --> 2020.6.24-0       anaconda
    certifi:         2020.6.20-py36h9f0ad1d_0 conda-forge --> 2020.6.20-py36_0  anaconda
    openssl:         1.1.1g-h516909a_0        conda-forge --> 1.1.1g-h

#### Fetching the postal codes from the city of Toronto

In [3]:
url  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
if page.status_code == 200:
    print('Page download successful')
else:
    print('Page download unsuccessful. Error code: {}'.format(page.status_code))

Page download successful


#### Transformation data into Panda dataframe and data cleansing


In [5]:
import lxml
df_html = pd.read_html(url, header=0, na_values = ['Not assigned'])[0]
df_html.head()

,Postal Code,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
df_html.dropna(subset=['Borough'], inplace=True)

In [8]:
n_empty_neighborhood = df_html[df_html['Neighborhood'].isna()].shape[0]
print('Number of rows on which Neighborhood column is empty: {}'.format(n_empty_neighborhood))

Number of rows on which Neighborhood column is empty: 0


#### Neighbourhood grouping data with postal codes


In [10]:
df_postcodes =pd.DataFrame( df_html.groupby(['Postal Code','Borough']).Neighborhood.agg([('Neighborhood', ', '.join)]))
df_postcodes.reset_index(inplace=True)
df_postcodes.head(5)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
df_html.dropna(subset=['Borough'], inplace=True)

In [12]:
print('The shape of the dataset is:',df_postcodes.shape)

The shape of the dataset is: (103, 3)


#### Identifying process of the Latitude and Longitude values of neigbourhoods Toronto city 

In [13]:
df_postcodes.to_csv('Toronto_Postcodes.csv')

In [14]:
url_csv = 'http://cocl.us/Geospatial_data'
df_coordinates = pd.read_csv(url_csv)
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
df_neighborhoods = pd.read_csv('Toronto_Postcodes.csv',index_col=[0])
df_neighborhoods.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [30]:
df_coordinates.rename(columns={'PostalCode': 'PostalCode'}, inplace=True)
df_neighborhoods.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

####  Merging the neighborhoods and coordinates in Toronto

In [31]:
df_neighborhoods_coordinates = pd.merge(df_neighborhoods, df_coordinates, on='PostalCode')
df_neighborhoods_coordinates.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [79]:
# Check coordinates for a couple of neighborhoods
df_neighborhoods_coordinates[(df_neighborhoods_coordinates['PostalCode']=='M5G') |
                             (df_neighborhoods_coordinates['PostalCode']=='M2H') ]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
17,M2H,North York,Hillcrest Village,43.803762,-79.363452
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [34]:
df_neighborhoods_coordinates.to_csv('Toronto_Postcodes_2.csv')

In [35]:
df = pd.read_csv('Toronto_Postcodes_2.csv', index_col=0)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [36]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [37]:
df.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)

In [38]:
df.groupby('Borough').count()['Neighborhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64

#### Creating the dataframe exclusively for Toronto City

In [39]:
df_toronto = df[df['Borough'].str.contains('Toronto')]
df_toronto.reset_index(inplace=True)
df_toronto.drop('index', axis=1, inplace=True)
df_toronto.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [40]:
print(df_toronto.groupby('Borough').count()['Neighborhood'])

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
West Toronto         6
Name: Neighborhood, dtype: int64


In [41]:
boroughs = df_toronto['Borough'].unique().tolist()

In [42]:
lat_toronto = df_toronto['Latitude'].mean()
lon_toronto = df_toronto['Longitude'].mean()
print('The geographical coordinates of Toronto are {}, {}'.format(lat_toronto, lon_toronto))

The geographical coordinates of Toronto are 43.66713498717948, -79.38987324871795


In [43]:
borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3))

In [44]:
map_toronto = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], 
                                           df_toronto['Longitude'],
                                           df_toronto['Borough'], 
                                           df_toronto['Neighborhood']):
    label_text = borough + ' - ' + neighborhood
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=borough_color[borough],
        fill_color=borough_color[borough],
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

#### Identifying the potentional venues around Toronto using Foursquare

In [45]:
CLIENT_ID = 'H0ZT0OPMNBLLYE5TVIVZUAGJ3A2W0U4EVFIOOHKANQ31313R' # your Foursquare ID
CLIENT_SECRET = '05ZZ5J20PDMSMRXHSUFVWTFSIAAIFOSOAADSDOOJWULEM3Y3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [47]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
   
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [48]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                latitudes=df_toronto['Latitude'],
                                longitudes=df_toronto['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [49]:
toronto_venues.shape

(1617, 7)

In [50]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",15,15,15,15,15,15
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,66,66,66,66,66,66
Christie,18,18,18,18,18,18
Church and Wellesley,74,74,74,74,74,74
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


In [51]:
toronto_venues['Venue Category'].unique()[:100]

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Cosmetics Shop', 'Greek Restaurant', 'Ice Cream Shop',
       'Italian Restaurant', 'Brewery', 'Fruit & Vegetable Store',
       'Yoga Studio', 'Juice Bar', 'Restaurant', 'Pizza Place',
       'Bookstore', 'Bubble Tea Shop', 'Dessert Shop',
       'Furniture / Home Store', 'Spa', 'Grocery Store', 'Coffee Shop',
       'Bakery', 'Caribbean Restaurant', 'Japanese Restaurant',
       'Indian Restaurant', 'Café', 'Lounge', 'Frozen Yogurt Shop',
       'Liquor Store', 'American Restaurant', 'Gym', 'Fish & Chips Shop',
       'Fast Food Restaurant', 'Sushi Restaurant', 'Park', 'Pet Store',
       'Steakhouse', 'Burrito Place', 'Movie Theater', 'Sandwich Place',
       'Board Shop', 'Food & Drink Shop', 'Fish Market', 'Gay Bar',
       'Thai Restaurant', 'Seafood Restaurant', 'Cheese Shop',
       'Comfort Food Restaurant', 'Middle Eastern Restaurant',
       'Stationery Store', 'Coworking Space', 'Wine Bar',
       'Latin Am

#### Clustering and segmenting  of restuarnts in Toronto to find the best location for an American Restaurant as per client requierement

In [52]:
"American Restaurant" in toronto_venues['Venue Category'].unique()

True

In [53]:
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhoods'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(1617, 235)


,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
to_grouped = to_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(to_grouped.shape)
to_grouped

(39, 235)


,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.066667,0.066667,0.133333,0.133333,0.133333,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.015152,0.000000,0.000000,0.015152,0.000000,0.000000,0.015152
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.013514,0.000000,0.000000,0.000000,0.000000,0.000000,0.013514,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.013514,0.000000,0.027027
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.030303,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [55]:
len(to_grouped[to_grouped["American Restaurant"] > 0])

10

In [56]:
to_American = to_grouped[["Neighborhoods","American Restaurant"]]

In [57]:
to_American.head(9)

,Neighborhoods,American Restaurant
0,Berczy Park,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000
2,"Business reply mail Processing Centre, South C...",0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000
4,Central Bay Street,0.000000
5,Christie,0.000000
6,Church and Wellesley,0.013514
7,"Commerce Court, Victoria Hotel",0.040000
8,Davisville,0.000000


#### K Means clustering model is used for Clustering to find the best location with less Amercian resturants

In [59]:
from sklearn.cluster import KMeans
toclusters = 5

to_clustering = to_American.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=1)
kmeans.fit_transform(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([1, 1, 1, 1, 1, 1, 4, 3, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1],
      dtype=int32)

In [60]:
to_merged = to_American.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_

In [61]:
to_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
to_merged.head(5)

,Neighborhood,American Restaurant,Cluster Labels
0,Berczy Park,0.0,1
1,"Brockton, Parkdale Village, Exhibition Place",0.0,1
2,"Business reply mail Processing Centre, South C...",0.0,1
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,1
4,Central Bay Street,0.0,1


In [66]:
to_merged = to_merged.join(toronto_venues.set_index("Neighborhood"), on="Neighborhood")

print(to_merged.shape)
to_merged.head()

(1617, 9)


,Neighborhood,American Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
36,"The Danforth West, Riverdale",0.023256,0,43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
36,"The Danforth West, Riverdale",0.023256,0,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
36,"The Danforth West, Riverdale",0.023256,0,43.679557,-79.352188,La Diperie,43.677702,-79.352265,Ice Cream Shop
36,"The Danforth West, Riverdale",0.023256,0,43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
36,"The Danforth West, Riverdale",0.023256,0,43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant


In [67]:
to_merged.sort_values(["Cluster Labels"], inplace=True)
to_merged.head(15)

,Neighborhood,American Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
36,"The Danforth West, Riverdale",0.023256,0,43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
11,"First Canadian Place, Underground city",0.030000,0,43.648429,-79.382280,Mos Mos Coffee,43.648159,-79.378745,Café
11,"First Canadian Place, Underground city",0.030000,0,43.648429,-79.382280,Walrus Pub & Beer Hall,43.647375,-79.379515,Pub
11,"First Canadian Place, Underground city",0.030000,0,43.648429,-79.382280,Dineen @CommerceCourt,43.648251,-79.380127,Coffee Shop
11,"First Canadian Place, Underground city",0.030000,0,43.648429,-79.382280,Cafe Landwer,43.648753,-79.385367,Café
11,"First Canadian Place, Underground city",0.030000,0,43.648429,-79.382280,DAVIDsTEA,43.646506,-79.380145,Tea Room
11,"First Canadian Place, Underground city",0.030000,0,43.648429,-79.382280,Indigospirit,43.648350,-79.380347,Bookstore
11,"First Canadian Place, Underground city",0.030000,0,43.648429,-79.382280,Kupfert & Kim (First Canadian Place),43.648547,-79.381624,Gluten-free Restaurant
11,"First Canadian Place, Underground city",0.030000,0,43.648429,-79.382280,Toronto PATH System,43.649903,-79.383053,General Travel
11,"First Canadian Place, Underground city",0.030000,0,43.648429,-79.382280,Ki Modern Japanese + Bar,43.647167,-79.379608,Japanese Restaurant


#### Identifying the locations of the Restaurnats in Toronto using folium Map

In [68]:
map_clusters = folium.Map(location=[lat_toronto, lon_toronto],zoom_start=14)

# set color scheme for the clusters


# add markers to the map
markers_colors={}
markers_colors[0] = 'red'
markers_colors[1] = 'blue'
markers_colors[2] = 'green'
markers_colors[3] = 'yellow'
markers_colors[4] = 'cyan'
markers_colors[5] = 'black'
for lat, lon, cluster in zip(to_merged['Neighborhood Latitude'], to_merged['Neighborhood Longitude'], to_merged['Cluster Labels']):
    
    
    folium.features.CircleMarker(
        [lat, lon],
        radius=5,
       
        color =markers_colors[cluster],
        fill_color=markers_colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Identifying American Restaurants in "Cluster 0"

In [69]:
#Cluster 0
to_merged.loc[(to_merged['Cluster Labels'] ==0) & (to_merged['Venue Category'] == 'American Restaurant') ]

,Neighborhood,American Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
11,"First Canadian Place, Underground city",0.030000,0,43.648429,-79.382280,Cactus Club Cafe,43.649552,-79.381671,American Restaurant
11,"First Canadian Place, Underground city",0.030000,0,43.648429,-79.382280,Richmond Station,43.651569,-79.379266,American Restaurant
11,"First Canadian Place, Underground city",0.030000,0,43.648429,-79.382280,The Gabardine,43.650988,-79.381225,American Restaurant
36,"The Danforth West, Riverdale",0.023256,0,43.679557,-79.352188,Factory Girl,43.676693,-79.356299,American Restaurant
37,"Toronto Dominion Centre, Design Exchange",0.030000,0,43.647177,-79.381576,Jump,43.648147,-79.378752,American Restaurant
37,"Toronto Dominion Centre, Design Exchange",0.030000,0,43.647177,-79.381576,The Gabardine,43.650988,-79.381225,American Restaurant
37,"Toronto Dominion Centre, Design Exchange",0.030000,0,43.647177,-79.381576,Cactus Club Cafe,43.649552,-79.381671,American Restaurant


#### Identfying American Restaurants in "Cluster 1"

In [70]:
#Cluster 1
to_merged.loc[(to_merged['Cluster Labels'] ==1) & (to_merged['Venue Category'] == 'American Restaurant') ]

,Neighborhood,American Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


#### Identifying American Restaurants in "Cluster 2"

In [71]:
#Cluster 2
to_merged.loc[(to_merged['Cluster Labels'] ==2) & (to_merged['Venue Category'] == 'American Restaurant') ]

,Neighborhood,American Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
32,Studio District,0.050000,2,43.659526,-79.340923,Brooklyn Tavern,43.661937,-79.335938,American Restaurant
33,"Summerhill West, Rathnelly, South Hill, Forest...",0.055556,2,43.686412,-79.400049,Union Social Eatery,43.687895,-79.394916,American Restaurant
32,Studio District,0.050000,2,43.659526,-79.340923,braised,43.660452,-79.343346,American Restaurant


#### Identifying American Restaurants in "Cluster 3"

In [72]:
#Cluster 3
to_merged.loc[(to_merged['Cluster Labels'] ==3) & (to_merged['Venue Category'] == 'American Restaurant') ]

,Neighborhood,American Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
29,St. James Town,0.038462,3,43.651494,-79.375418,Richmond Station,43.651569,-79.379266,American Restaurant
29,St. James Town,0.038462,3,43.651494,-79.375418,Jump,43.648147,-79.378752,American Restaurant
29,St. James Town,0.038462,3,43.651494,-79.375418,The Gabardine,43.650988,-79.381225,American Restaurant
7,"Commerce Court, Victoria Hotel",0.040000,3,43.648198,-79.379817,Richmond Station,43.651569,-79.379266,American Restaurant
7,"Commerce Court, Victoria Hotel",0.040000,3,43.648198,-79.379817,Cactus Club Cafe,43.649552,-79.381671,American Restaurant
7,"Commerce Court, Victoria Hotel",0.040000,3,43.648198,-79.379817,The Gabardine,43.650988,-79.381225,American Restaurant
7,"Commerce Court, Victoria Hotel",0.040000,3,43.648198,-79.379817,Jump,43.648147,-79.378752,American Restaurant


#### Identifying American Restaurants in "Cluster 4"

In [73]:
#Cluster 4
to_merged.loc[(to_merged['Cluster Labels'] ==4) & (to_merged['Venue Category'] == 'American Restaurant') ]

,Neighborhood,American Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
31,Stn A PO Boxes,0.010309,4,43.646435,-79.374846,Jump,43.648147,-79.378752,American Restaurant
6,Church and Wellesley,0.013514,4,43.665860,-79.383160,The Blake House,43.664468,-79.377471,American Restaurant
25,"Richmond, Adelaide, King",0.010638,4,43.650571,-79.384568,Richmond Station,43.651569,-79.379266,American Restaurant



# Conclusion

Most of the American restaurants are in cluster 0,cluster 3 and cluster 4 which is around
St.James town, Stn A PO Boxes, Church and Wellesley, Richmond, Adelaide, King,Commerce Court, Victoria Hotel,First Canadian Place, Underground city,Toronto Dominion Centre, Design Exchange.

And lowest in Cluster 1, Cluster 2 and Cluster 4 areas  there are good opportunities to open Richmond, Adelaide, King, Church and Wellesley 	Summerhill West, Rathnelly, South Hill,Studio District. 	

Looking at nearby venues it seems cluster 1 might be a good location as there are no American restaurants in these areas. Therefore, this project recommends the entrepreneur to open an authentic American restaurant in these locations.
